In [24]:
from elasticsearch import Elasticsearch

In [25]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
# To use the nested query, your index must include a nested field mapping. For example:
body = {
    "mappings": {
        "properties": {
            "driver": {
                "type": "nested",
                "properties": {
                    "last_name": {
                        "type": "text"
                    },
                    "vehicle": {
                        "type": "nested",
                        "properties": {
                            "make": {
                                "type": "text"
                            },
                            "model": {
                                "type": "text"
                            }
                        }
                    }
                }
            }
        }
    }
}
es.indices.create(index='es0', body=body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [26]:
es.index(index='es0', id=0, body={
    "driver": {
        "last_name": "McQueen",
        "vehicle": [
            {
                "make": "Powell Motors",
                "model": "Canyonero"
            },
            {
                "make": "Miller-Meteor",
                "model": "Ecto-1"
            }
        ]
    }
})
es.index(index='es0', id=1, body={
    "driver": {
        "last_name": "Hudson",
        "vehicle": [
            {
                "make": "Mifune",
                "model": "Mach Five"
            },
            {
                "make": "Miller-Meteor",
                "model": "Ecto-1"
            }
        ]
    }
})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [27]:
body = {
    "query": {
        "nested": {
            "path": "driver",
            "query": {
                "nested": {
                    "path": "driver.vehicle",
                    "query": {
                        "bool": {
                            "must": [
                                # You can now use a multi-level nested query to match documents based on the make and model fields.
                                {"match": {"driver.vehicle.make": "Powell Motors"}},
                                {"match": {"driver.vehicle.model": "Canyonero"}}
                            ]
                        }
                    }
                }
            }
        }
    }
}
# The nested query searches nested field objects as if they were indexed as separate documents. If an object matches the search, the nested query returns the root parent document.
'''
GET test/_search
`body`
'''
es.search(index='es0', body=body)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 3.7349272,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 3.7349272,
    '_source': {'driver': {'last_name': 'McQueen',
      'vehicle': [{'make': 'Powell Motors', 'model': 'Canyonero'},
       {'make': 'Miller-Meteor', 'model': 'Ecto-1'}]}}}]}}

In [28]:
es.indices.delete('es0')

{'acknowledged': True}